In [26]:
import IP2Location
import csv,json
import pathlib
import requests
from urllib.request import urlopen
import pandas as pd
from git import Repo
import os
import sqlite3
r = Repo('/home/dakes/blacklisted_ipsets/')
import re
import datetime
from datetime import date,timedelta
import sqlite3
from time import strptime
plus = "^\+\d{1}"
minus = "^\-\d{1}"
IP2LocObj = IP2Location.IP2Location()
IP2LocObj.open('/home/dakes/Downloads/Jupyter_scripts/IP2LOCATION-LITE-DB5.BIN')
ff=[] 
allo=[]
x = datetime.datetime.now()
today= date(x.year,x.month,x.day)
print(today)

2021-07-07


In [2]:
tmp = "Sun Feb 25 11:00:54 UTC 2018".split(' ')
strptime(tmp[1],'%b').tm_mon

2

In [3]:
#daily fetch
!git fetch --all

Fetching origin
remote: Enumerating objects: 1455, done.
remote: Counting objects: 100% (1455/1455), done.
remote: Compressing objects: 100% (305/305), done.
remote: Total 1455 (delta 1150), reused 1455 (delta 1150), pack-reused 0
Receiving objects: 100% (1455/1455), 3.64 MiB | 2.18 MiB/s, done.
Resolving deltas: 100% (1150/1150), completed with 256 local objects.
From https://github.com/firehol/blocklist-ipsets
 + 75def9c9...e911cf59 master     -> origin/master  (forced update)
 + 849310f7...d26f80de gh-pages   -> origin/gh-pages  (forced update)


In [10]:
fnames=[]
for path in pathlib.Path("/home/dakes/blacklisted_ipsets").iterdir():
    if str(path).endswith('.netset') or str(path).endswith('.ipset'):
        df = pd.read_csv(path,sep=";ofi",index_col=False,engine='python')
        for line in df.iterrows():
                if "# Category" in line[1][0] :
                            if "malware" not in line[1][0] and "spam" not in line[1][0] and "attacks" not in line[1][0]:  
                                break
                            else:
                                fnames.append(str(path).replace("/home/dakes/blacklisted_ipsets/",""))
#print((fnames))

In [ ]:
        with open("/home/dakes/description.json",'r') as jsf:
            descr = json.load(jsf,strict = False)
        files=[]
        removed=[]
    #with open('blacklisted_ipsets.csv', 'a', newline='') as csvfile:
        #spamwriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        #spamwriter.writerow(['Name','Maintainer','Maintainer URL','Tracker List Url','Source File Date','File Update','Last update','IPs'])#+x.month*100 + x.day)
        for path in pathlib.Path("/home/dakes/blacklisted_ipsets").iterdir():
            flag =False
            if str(path).endswith('.netset') or str(path).endswith('.ipset'):
                diff_cut=[]
                df = pd.read_csv(path,sep=";ofi",index_col=False,engine='python')
                name=''
                maintainer=''
                maintainerURL=''
                listSourceUrl=''
                sourceFileDate=''
                recentFileUpdate=''
                for line in df.iterrows():
                        if "# Category" in line[1][0] :
                            if "malware" not in line[1][0] and "spam" not in line[1][0] and "attacks" not in line[1][0]:  
                                break
                            else:
                                category = line[1][0].replace("# Category      : ","")
                        if "# Maintainer      :" in line[1][0]:
                            maintainer = line[1][0].replace("# Maintainer      : ","")
                        if "# Maintainer URL  :" in line[1][0]:
                            maintainerURL = line[1][0].replace("# Maintainer URL  : ","")
                        if "# List source URL :" in line[1][0]:
                            listSourceUrl = line[1][0].replace("# List source URL : ","")
                        if "# Source File Date:" in line[1][0]:
                            sourceFileDate = line[1][0].replace("# Source File Date: ","")
                                
                        if "# This File Date  :" in line[1][0]:
                            year=0
                            timestamp=''
                            recentFileUpdate = line[1][0].replace("# This File Date  : ","")
                            name = str(path).replace('/home/dakes/blacklisted_ipsets/','')
                            timestamp=recentFileUpdate.split(' ')
                            timestamp=list(filter(None,timestamp))
                            
                            day=int(timestamp[2])
                            month = strptime(timestamp[1],'%b').tm_mon
                            year = int(timestamp[5])

                            anyday = date(year,month,day)
                            
                            print(today-anyday)
                            if today-anyday <=timedelta(days=40) :
                                flag=True
                            else:
                                flag=False
                                break
                            
                            
                            diff = r.git.diff("@{upstream}",name)
                            source = str(path)
                            des = source.replace("/home/dakes/blacklisted_ipsets/","")
                            des = des.replace('.ipset',"")
                            des = des.replace('.netset',"")
                            for line in diff.splitlines():
                                if re.findall(plus,line): #create today's list
                                    line = line.split('/',1)[0]
                                    line = ".".join(line.split('.')[0:-1])
                                    line = line +'.1'
                                    diff_cut.append(line.replace('+',""))
                                    #elif re.findall(minus,line): #removes last day of the week and (-) of other days
                                    #diff_cut.append(line.replace('+',""))
                                    #diff_cut.append(line.replace('-',""))
                                    #diff_cut.append(line)
                                    #for element in data:
                                    #    if datetime.datetime.now() - element['properties']['datetime'] > 7:
                                    #        element.pop()
                                    #    if element['properties']['ip']== line:
                                    #        element.pop()
                                    #diff_cut.append(line.replace('-',""))
                                    #spamwriter.writerow([name.replace('/home/dakes/blacklisted_ipsets/',''),maintainer,maintainerURL,listSourceUrl,sourceFileDate,recentFileUpdate,(x.month*100+x.day),','.join(diff_cut)])
                            break      
                if  flag:
                    print(flag,recentFileUpdate)
                    files.append((name,maintainer,maintainerURL,listSourceUrl,sourceFileDate,recentFileUpdate,diff_cut,descr[des]))
                    
print(len(files))


27 days, 0:00:00
True Thu Jun 10 10:00:35 UTC 2021
27 days, 0:00:00
True Thu Jun 10 06:12:13 UTC 2021
27 days, 0:00:00
True Thu Jun 10 10:20:04 UTC 2021
1228 days, 0:00:00
343 days, 0:00:00
27 days, 0:00:00
True Thu Jun 10 02:56:11 UTC 2021
577 days, 0:00:00
295 days, 0:00:00
1126 days, 0:00:00
577 days, 0:00:00
1126 days, 0:00:00
27 days, 0:00:00
True Thu Jun 10 10:20:05 UTC 2021
1126 days, 0:00:00
1043 days, 0:00:00
281 days, 0:00:00
1126 days, 0:00:00
27 days, 0:00:00
True Thu Jun 10 00:28:03 UTC 2021
978 days, 0:00:00
27 days, 0:00:00
True Thu Jun 10 05:48:05 UTC 2021
27 days, 0:00:00
True Thu Jun 10 04:36:11 UTC 2021
1126 days, 0:00:00
343 days, 0:00:00
27 days, 0:00:00
True Thu Jun 10 04:36:11 UTC 2021
376 days, 0:00:00
281 days, 0:00:00
936 days, 0:00:00
282 days, 0:00:00
1126 days, 0:00:00
1043 days, 0:00:00
1126 days, 0:00:00
537 days, 0:00:00
382 days, 0:00:00
1126 days, 0:00:00
1126 days, 0:00:00
27 days, 0:00:00
True Thu Jun 10 04:36:11 UTC 2021
1044 days, 0:00:00
281 days,

In [3]:
'''def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)

    return conn


def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
'''

'def create_connection(db_file):\n    """ create a database connection to the SQLite database\n        specified by db_file\n    :param db_file: database file\n    :return: Connection object or None\n    """\n    conn = None\n    try:\n        conn = sqlite3.connect(db_file)\n        return conn\n    except Error as e:\n        print(e)\n\n    return conn\n\n\ndef create_table(conn, create_table_sql):\n    """ create a table from the create_table_sql statement\n    :param conn: Connection object\n    :param create_table_sql: a CREATE TABLE statement\n    :return:\n    """\n    try:\n        c = conn.cursor()\n        c.execute(create_table_sql)\n    except Error as e:\n        print(e)\n'

In [ ]:
#after fetching and storing re-init repo to original state
!git reset --hard origin/master


In [ ]:
'''
database = r"SQLite_Python.db"

sql_create_projects_table = """ CREATE TABLE IF NOT EXISTS ipsets (
                                        id integer PRIMARY KEY,
                                        name text NOT NULL,
                                        begin_date text,
                                        end_date text
                                    ); """
conn = create_connection(database)

# create tables
if conn is not None:
    create_table(conn, sql_create_projects_table)

sqliteConnection = sqlite3.connect('SQLite_Python.db',
                                           detect_types=sqlite3.PARSE_DECLTYPES |
                                                        sqlite3.PARSE_COLNAMES)
cursor = sqliteConnection.cursor()
x = datetime.datetime.now()
print(x.year*10000+x.month*100 + x.day)
print(x.year)
#cursor.execute("INSERT INTO ips VALUES (?,?,?,?,?)",(x,'192.168.1.1',"zeus.ipset","ipblock","https://github.com"))
#sqliteConnection.commit()
for row in cursor.execute('SELECT * FROM projects'):
    print(row)

cursor.close()
except sqlite3.Error as error:
    print("Error while working with SQLite", error)
finally:
    if (sqliteConnection):
        sqliteConnection.close()
        print("sqlite connection is closed")
'''

In [38]:
for i in files:
    print(i)

('bambenek_suppobox.ipset', 'Bambenek Consulting', 'http://osint.bambenekconsulting.com/feeds/', 'http://osint.bambenekconsulting.com/feeds/suppobox-iplist.txt', 'Wed Jul 29 20:19:32 UTC 2020', 'Wed Jul 29 20:24:08 UTC 2020', [], 'Bambenek Consulting feed of current IPs of suppobox C&Cs with 90 minute lookback')
('ransomware_torrentlocker_c2.ipset', 'Abuse.ch', 'https://ransomwaretracker.abuse.ch', 'https://ransomwaretracker.abuse.ch/downloads/TL_C2_IPBL.txt', 'Sun Dec  8 11:16:09 UTC 2019', 'Sun Dec  8 11:16:09 UTC 2019', [], 'Abuse.ch Ransomware Tracker Ransomware Tracker tracks and monitors the status of domain names')
('lashback_ubl.ipset', 'The LashBack Unsubscribe Blacklist', 'http://blacklist.lashback.com/', 'http://www.unsubscore.com/blacklist.txt', 'Mon Sep 14 18:00:11 UTC 2020', 'Tue Sep 15 00:12:34 UTC 2020', [], 'The LashBack UBL The Unsubscribe Blacklist (UBL) is a real-time blacklist of IP addresses which are sending email to names harvested from suppression files (this i

In [28]:
features=[]
for i in files:
    if not i[6] or i[6]=='127.0.0.1' or i[6]=='192.168.1.1':
        continue
    else:
        for ip in i[6]:
            rec =  IP2LocObj.get_all(ip)
            #print(".".join(ip.split('.')[0:-1])+'.*')
            features.append(({
                                  "type": "Feature",
                                  "properties": {
                                    "location": rec.city,
                                    "source":"https://github.com/firehol/blocklist-ipsets/blob/master/"+i[0].replace('/home/dakes/blacklisted_ipsets/',''),
                                    "ip": ".".join(rec.ip.split('.')[0:-1])+'.*',
                                    "maintainer":i[1],
                                    "category" : "malware",
                                    "description": i[7],
                                    "alt1": "extreme-ip-lookup.com/"+rec.ip,
                                    "alt2":"talosintelligence.com/reputation_center/lookup?search="+rec.ip
                                    #"0datetime":i[7]
                                  },
                                  "geometry": {
                                    "type": "Point",
                                    "coordinates": [float(rec.longitude), float(rec.latitude)]
                                  }
                                }))
#len(features)
#del features[1000:]
#len(features)


In [29]:
len(features)

0

In [32]:
with open("example1000.json","w") as f: #edit your  path
    json.dump(features,f,indent=4) 
f.close()

In [53]:
'''import datetime
import sqlite3

def addDeveloper(id, name, joiningDate):
    try:
        sqliteConnection = sqlite3.connect('SQLite_Python.db',
                                           detect_types=sqlite3.PARSE_DECLTYPES |
                                                        sqlite3.PARSE_COLNAMES)
        cursor = sqliteConnection.cursor()
        print("Connected to SQLite")

        cursor = sqliteConnection.cursor()
        cursor.execute(sqlite_create_table_query)

        # insert developer detail
        sqlite_insert_with_param = """INSERT INTO 'new_developers'
                          ('id', 'name', 'joiningDate') 
                          VALUES (?, ?, ?);"""

        data_tuple = (id, name, joiningDate)
        cursor.execute(sqlite_insert_with_param, data_tuple)
        sqliteConnection.commit()
        print("Developer added successfully \n")

        # get developer detail
        sqlite_select_query = """SELECT name, joiningDate from new_developers where id = ?"""
        cursor.execute(sqlite_select_query, (1,))
        records = cursor.fetchall()

        for row in records:
            developer = row[0]
            joining_Date = row[1]
            print(developer, " joined on", joiningDate)
            print("joining date type is", type(joining_Date))

        cursor.close()

    except sqlite3.Error as error:
        print("Error while working with SQLite", error)
    finally:
        if (sqliteConnection):
            sqliteConnection.close()
            print("sqlite connection is closed")

addDeveloper(1, 'Kostas', datetime.datetime.now())
'''

Connected to SQLite
Error while working with SQLite table new_developers already exists
sqlite connection is closed
